In [2]:
import pandas as pd
import datetime as dt
import urllib
import zipfile
import re
from geopy.distance import vincenty

In [3]:
url = 'http://web.mta.info/developers/data/nyct/subway/google_transit.zip'
urllib.urlretrieve (url, "GTFS_nyc_Subway.zip")

('GTFS_nyc_Subway.zip', <httplib.HTTPMessage instance at 0x106d78ea8>)

In [4]:
zip_ref = zipfile.ZipFile("GTFS_nyc_Subway.zip", 'r')
zip_ref.extractall("GTFS_nyc_Subway")
zip_ref.close()

In [5]:
stop_times= pd.read_csv('GTFS_nyc_Subway/stop_times.txt')

In [6]:
stop_times.head(2)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,A20171105SAT_036000_GS.N01R,06:00:00,06:00:00,901N,1,NaN,0,0,NaN
1,A20171105SAT_036000_GS.N01R,06:01:30,06:01:30,902N,2,NaN,0,0,NaN


In [7]:
stop_times['service_id']=stop_times['trip_id'].apply(lambda x: x.split('_')[0])
stop_times['sub_trip_id']=stop_times['trip_id'].apply(lambda x: x.split('_')[1])
stop_times['train+direction']=stop_times['trip_id'].apply(lambda x: x.split('_')[2])
del stop_times['stop_headsign']
del stop_times['pickup_type']
del stop_times['drop_off_type']
del stop_times['shape_dist_traveled']

In [8]:
stop_times['train'] = stop_times['train+direction'].apply(lambda x: x.split('.')[0])
stop_times['day'] = stop_times['service_id'].apply(lambda x: x[-3:])

In [9]:
stop_times.loc[stop_times[stop_times['train+direction'].isin(['N..N72R','N..S72R'])].index,['train']]='W'

In [10]:
stop_train= stop_times[['stop_id','train']].drop_duplicates()

In [11]:
stop_train['stop_id'] = map(lambda x:x[:-1],stop_train['stop_id'])

In [12]:
stop_train = stop_train.drop_duplicates().sort_values(['train','stop_id']).reset_index(drop= True)

In [13]:
set(stop_train.train)

{'1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'FS',
 'G',
 'GS',
 'H',
 'J',
 'L',
 'M',
 'N',
 'Q',
 'R',
 'SI',
 'W'}

In [14]:
stops=pd.read_csv('GTFS_nyc_Subway/stops.txt')

In [15]:
stop_train = stop_train.merge(stops[['stop_id','stop_name','stop_lat','stop_lon']],on='stop_id',how='left')

In [17]:
stop_train.to_csv('stop_train.csv')

In [19]:
stop_train.head()

,stop_id,train,stop_name,stop_lat,stop_lon
0,101,1,Van Cortlandt Park - 242 St,40.889248,-73.898583
1,103,1,238 St,40.884667,-73.900870
2,104,1,231 St,40.878856,-73.904834
3,106,1,Marble Hill - 225 St,40.874561,-73.909831
4,107,1,215 St,40.869444,-73.915279


In [24]:
stop_times['stop_id'] = stop_times['stop_id'].str[:-1]

In [25]:
stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,service_id,sub_trip_id,train+direction,train,day
0,A20171105SAT_036000_GS.N01R,06:00:00,06:00:00,901,1,A20171105SAT,036000,GS.N01R,GS,SAT
1,A20171105SAT_036000_GS.N01R,06:01:30,06:01:30,902,2,A20171105SAT,036000,GS.N01R,GS,SAT
2,A20171105SAT_036400_GS.S01R,06:04:00,06:04:00,902,1,A20171105SAT,036400,GS.S01R,GS,SAT
3,A20171105SAT_036400_GS.S01R,06:05:30,06:05:30,901,2,A20171105SAT,036400,GS.S01R,GS,SAT
4,A20171105SAT_037000_GS.N01R,06:10:00,06:10:00,901,1,A20171105SAT,037000,GS.N01R,GS,SAT
